In [1]:
#library and code setup
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt
!pip install newspaper3k
!pip install mpi4py

     |████████████████████████████████| 217.8MB 56kB/s 
     |████████████████████████████████| 204kB 48.4MB/s 
     |████████████████████████████████| 778kB 7.9MB/s 
     |████████████████████████████████| 215kB 9.4MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 7.4MB 22.0MB/s 
     |████████████████████████████████| 194kB 48.9MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13539 sha256=1f4bd16dbbab551d7bd926674b51846242d880e084e1f0d7e4655c552553c319
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=1b13c7fc8ea80c9e3b584bf20752ab2d53fda44cca8f5d0f98eee3da043c2491
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl s

In [0]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

In [0]:
queryURLbase = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=(Wildlife%20Conservation%20Or%20pollution%20Or%20poaching%20Or%20hazardous%20waste)%20sourcelang:eng&mode=artlist&maxrecords=250&sort=hybridrel"


In [4]:
# these remove files from previous runs
!rm -rf articles
!rm *.csv

rm: cannot remove '*.csv': No such file or directory


###API

In [14]:
from datetime import date, timedelta
import pandas as pd
import os
import urllib.request

# generic functions to pull and write data to disk based on date
def get_filename(x):
  temp = x.strftime("%Y%m%d_%H%M%S")
  # date = temp[0].strftime('%Y%m%d')
  temp.replace("[","")
  temp.replace("]","")
  temp.replace(":","")
  return "{}_gdeltdata.csv".format(temp)

def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          time = filename.split("_")[1]
          time = time.replace(':', '')
          startdate = date+time #staring hour
          enddate = date+time[:2]+"5959" #adds 59:59 on to the hour to get full day
          query = queryURLbase + "&startdatetime=" + startdate + "&enddatetime=" + enddate
          with urllib.request.urlopen(query) as testfile, open(filename, 'w') as f:
            f.write(testfile.read().decode())
    except:
        print("Error occurred")

daterange = sc.parallelize(pd.date_range('2020 Jan 1','2020 Jan 5',freq ='1H'))

dates = daterange.map(get_filename)
dbg(dates)
dates.foreach(intofile)

['20200101_000000_gdeltdata.csv', '20200101_010000_gdeltdata.csv', '20200101_020000_gdeltdata.csv', '20200101_030000_gdeltdata.csv', '20200101_040000_gdeltdata.csv', '20200101_050000_gdeltdata.csv', '20200101_060000_gdeltdata.csv', '20200101_070000_gdeltdata.csv', '20200101_080000_gdeltdata.csv', '20200101_090000_gdeltdata.csv', '20200101_100000_gdeltdata.csv', '20200101_110000_gdeltdata.csv', '20200101_120000_gdeltdata.csv', '20200101_130000_gdeltdata.csv', '20200101_140000_gdeltdata.csv', '20200101_150000_gdeltdata.csv', '20200101_160000_gdeltdata.csv', '20200101_170000_gdeltdata.csv', '20200101_180000_gdeltdata.csv', '20200101_190000_gdeltdata.csv', '20200101_200000_gdeltdata.csv', '20200101_210000_gdeltdata.csv', '20200101_220000_gdeltdata.csv', '20200101_230000_gdeltdata.csv', '20200102_000000_gdeltdata.csv', '20200102_010000_gdeltdata.csv', '20200102_020000_gdeltdata.csv', '20200102_030000_gdeltdata.csv', '20200102_040000_gdeltdata.csv', '20200102_050000_gdeltdata.csv', '20200102

In [19]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
print(dates.collect())
data_urls = sqlContext.read.option("header", "true").csv(dates.collect())
# df_dupfix_api = data_urls.dropDuplicates(['URL'])
# data_urls = data_urls.dropDuplicates(['URL'])
df_dupfix_api.write.csv("hell2o.csv")

['20200101_000000_gdeltdata.csv', '20200101_010000_gdeltdata.csv', '20200101_020000_gdeltdata.csv', '20200101_030000_gdeltdata.csv', '20200101_040000_gdeltdata.csv', '20200101_050000_gdeltdata.csv', '20200101_060000_gdeltdata.csv', '20200101_070000_gdeltdata.csv', '20200101_080000_gdeltdata.csv', '20200101_090000_gdeltdata.csv', '20200101_100000_gdeltdata.csv', '20200101_110000_gdeltdata.csv', '20200101_120000_gdeltdata.csv', '20200101_130000_gdeltdata.csv', '20200101_140000_gdeltdata.csv', '20200101_150000_gdeltdata.csv', '20200101_160000_gdeltdata.csv', '20200101_170000_gdeltdata.csv', '20200101_180000_gdeltdata.csv', '20200101_190000_gdeltdata.csv', '20200101_200000_gdeltdata.csv', '20200101_210000_gdeltdata.csv', '20200101_220000_gdeltdata.csv', '20200101_230000_gdeltdata.csv', '20200102_000000_gdeltdata.csv', '20200102_010000_gdeltdata.csv', '20200102_020000_gdeltdata.csv', '20200102_030000_gdeltdata.csv', '20200102_040000_gdeltdata.csv', '20200102_050000_gdeltdata.csv', '20200102

###Normal webscraping

In [0]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()


# generic functions to pull and write data to disk based on date
def get_filename_web(x):
  date = x.strftime('%Y%m%d')
  return "{}_webscrapegdeltdata.csv".format(date)

def intofile_web(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='events',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
    except:
        print("Error occurred")

# pull the data from gdelt into multi files; this may take a long time
webscrape_dates_before = [get_filename_web(x) for x in pd.date_range('2020 Jan 1','2020 Jan 2')]
results = list(e.map(intofile_web,webscrape_dates_before))

In [8]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
data_webscrape = sqlContext.read.option("header", "true").csv(webscrape_dates_before)
data_webscrape = data_webscrape.withColumnRenamed("SOURCEURL", "URL")
print(data_webscrape.count())
data_webscrape.show()

1781
+-------------+--------+---------+----+------------+----------+-------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+-------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+--------------------+-------------+-------------+---------+--------------+-----------+----------+-----------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+----

###Join data

In [9]:
#Combine = data_webscrape.join(df_dupfix_api, data_webscrape.col("URL").equalTo(df_dupfix_api.col("URL")))
Combine = data_webscrape.join(df_dupfix_api, on=['URL'], how='left')
print(Combine.count())
Combine = Combine.dropDuplicates(['URL'])
print(Combine.count())
Combine.show()
#Try ActorGeo_CountryCode

1781
415
+--------------------+-------------+--------+---------+----+------------+----------+------------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+----------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+--------------------+-------------+-------------+---------+--------------+-----------+----------+-----------+-------------------+--------------+--------------------+---------------------+------------------+------------------+------------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+------------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+------------------+------

###Webscrap

In [10]:
combine_rdd = Combine.rdd.map(list)
dbg(combine_rdd)

[['https://sierranewsonline.com/breaking-biomass-project-closes-financing-plant-construction-to-start-in-april/', '896355585', '19200102', '192001', '1920', '1920.0055', 'CVL', 'COMMUNITY', None, None, None, None, None, 'CVL', None, None, None, None, None, None, None, None, None, None, None, None, '1', '031', ' Express intent to engage in material cooperation,  not specified below', '031', '03', '1', '5.2', '10', '1', '10', '0.6079027355623101', '3', 'Phoenix, Arizona, United States', 'US', 'USAZ', 'AZ013', '33.4484', '-112.074', '44784', '0', None, None, None, None, None, None, None, '3', 'Phoenix, Arizona, United States', 'US', 'USAZ', 'AZ013', '33.4484', '-112.074', '44784', '20200102234500', None, None, None], ['https://www.click2houston.com/news/local/2020/01/01/memorial-village-er-offers-free-flu-vaccines-for-first-two-weeks-of-january/', '896176163', '19200101', '192001', '1920', '1920.0027', 'HLH', 'DOCTOR', None, None, None, None, None, 'HLH', None, None, None, None, None, Non

In [11]:

import time
import requests
start = time.time()
print("start")
print(requests.get("https://sierranewsonline.com/breaking-biomass-project-closes-financing-plant-construction-to-start-in-april/").content)
end = time.time()
print(end - start)



start
b'<!DOCTYPE html>\n<html lang="en"\n\tprefix="og: https://ogp.me/ns#"  prefix="og: http://ogp.me/ns#">\n<head>\n<meta charset="UTF-8" />\n<title>North Fork Biomass Project Closes Financing, Plant Construction to Start in April | Sierra News Online</title>\n<link rel="profile" href="https://gmpg.org/xfn/11" />\n<link rel="pingback" href="https://sierranewsonline.com/xmlrpc.php" />\n<meta name="msvalidate.01" content="D61EBD71B7527D555553DB857654621B" />\n<script async=\'async\' src=\'https://www.googletagservices.com/tag/js/gpt.js\'></script>\n<script>\n  var googletag = googletag || {};\n  googletag.cmd = googletag.cmd || [];\n</script>\n<script>\ngoogletag.cmd.push(function() {\ngoogletag.defineSlot(\'/14996645/001\', [728, 90], \'div-gpt-ad-1483895676977-0\').addService(googletag.pubads());\ngoogletag.defineSlot(\'/14996645/002\', [300, 250], \'div-gpt-ad-1483907347097-0\').addService(googletag.pubads());\ngoogletag.defineSlot(\'/14996645/003\', [300, 250], \'div-gpt-ad-1483907

In [12]:
import requests

def get_article_content(url):
  print(url)
  #page = requests.get(url)
  #return(page.content)
  return 1

Content_baskets = combine_rdd.map(lambda x: (get_article_content(x[0]),x[39]))
dbg(Content_baskets)

[(1, 'US'), (1, 'US'), (1, 'US'), (1, 'US'), (1, 'US'), (1, None), (1, 'US'), (1, None), (1, 'EI'), (1, 'IS'), (1, 'IZ'), (1, 'NU'), (1, 'US'), (1, 'US'), (1, 'US'), (1, None), (1, 'GH'), (1, 'US'), (1, None), (1, 'IN'), (1, 'CH'), (1, 'US'), (1, None), (1, 'IZ'), (1, 'GA'), (1, None), (1, 'US'), (1, 'US'), (1, None), (1, None), (1, 'US'), (1, None), (1, 'US'), (1, 'CH'), (1, 'FR'), (1, None), (1, 'NZ'), (1, None), (1, 'US'), (1, None), (1, 'US'), (1, 'US'), (1, 'US'), (1, 'IN'), (1, 'LY'), (1, 'UK'), (1, 'US'), (1, 'US'), (1, None), (1, 'US'), (1, None), (1, 'MX'), (1, 'IN'), (1, 'AU'), (1, 'US'), (1, None), (1, None), (1, 'US'), (1, None), (1, 'US'), (1, 'IR'), (1, 'US'), (1, None), (1, 'US'), (1, 'US'), (1, 'US'), (1, None), (1, 'MY'), (1, 'CB'), (1, 'US'), (1, None), (1, 'AU'), (1, None), (1, 'SP'), (1, 'US'), (1, 'PK'), (1, 'US'), (1, None), (1, 'US'), (1, 'IN'), (1, 'UK'), (1, None), (1, None), (1, 'US'), (1, 'US'), (1, None), (1, None), (1, 'US'), (1, 'AS'), (1, 'UK'), (1, 'US')